In [ ]:
%matplotlib widget
import numpy as np
from BigM import BigM
from BigM import graph
from BigM import InputHandling
import random
mem = [
    np.array([ [1, 3, 3], [-2, 5, 1], [2, 8, 0], [-1, 1, 1] ]),
    np.array([ [1, 3, 4], [-1, 50, 9], [6, 50, 0], [-1, 1, -1] ]),
]

#graph.drawGraph(mem[1])
class N1(Exception):
    pass
class N2(Exception):
    pass

def handle_display_graph(prob : np.ndarray):
    while True:
        c = input("Voulez vous afficher un représentation graphique? (o/n)\n")
        if c in ['o', 'n']:
            break
    if c == 'o' :
        graph.drawGraph(prob) # TODO ipynb magik

def validate_with_msgs(expr: str) -> bool:
    if InputHandling.validateUserInput(expr):
        if InputHandling.validateUserInputSemantic(expr):
            return True
        else:
            print("l\'expression semble etre correcte mais n\'a pas de sens\n")
            return False
    else:
        print("expression fausse\n")
        return False

def handle_problem_input() -> np.ndarray:
    # fonction objectif
    while True:
        obj = input("Donner votre fct objective sous la forme: (max/min) z = ax +/- by ...\n")
        if validate_with_msgs(obj):
            break
    
    # Contraintes
    ctr = []
    print("Donner vos contraintes sous la forme ax +/- by ... ( <= / < / = / >= / > ) d \n")
    c = "go"
    while c :
        c = input(f"Donner la contrainte no {len(ctr)+1}, envoyer une chaine vide pour terminer\n")
        if c :
            if validate_with_msgs(c):
                ctr.append(c)
    
    return np.array(InputHandling.formatUserInput([obj] + ctr))

def appliquer_bigm(prob : np.ndarray , is_R2: bool):
    bm = BigM.BigM()
    iterations = bm.runBigM(prob)


def do_job(prob : np.ndarray , is_R2: bool) :
    if is_R2:
        handle_display_graph(prob)
    #appliquer bigm


def handle_choice():
    match input("votre choix ... ") :
        case '1':
            # assuming that all problems are R2
            prob = np.array(InputHandling.formatUserInput(random.choice(mem)))
            do_job(prob, True)
            
        case '2':
            prob = handle_problem_input()
            do_job(prob, True)
            
        case '3':
            prob = handle_problem_input()
            do_job(prob, False)
            
        case '4':
            print("Bye!")
            exit(0)
            
        case _:
            print("Choisir de 1 a 4\n")
            raise N1()

            
while True :
    print("""1- choisir programme de la memoire
            2- saisir programme dans R2
            3- saisir programme dans R3
            4- quitter\n""")
    try :
        handle_choice()
    except Exception:
        continue